# ✨ Camada Gold - Visões Analíticas do Domínio Cripto

## 📌 Objetivo
Este notebook tem como objetivo construir visões analíticas baseadas na camada Silver unificada (`cripto`) com foco em análises exploratórias e criação de *views* que permitam responder a perguntas de negócio e embasar decisões estratégicas.

## 🔍 O que será feito:
- Criação de *views temporárias* com base nos dados históricos e intradiários de criptoativos.
- Geração de gráficos para visualização de tendências e comportamento dos ativos ao longo do tempo.
- Identificação de padrões, picos de preço e variações diárias para facilitar análises.

## 📈 Visões a serem desenvolvidas:
1. **📊 Evolução do Preço por Ativo ao Longo do Tempo**  
   Linha do tempo com o preço em USD para cada ativo.

2. **📉 Variação Diária (%) por Ativo**  
   Cálculo da variação percentual diária do preço.

3. **🔥 Top 5 Maiores Altas e Quedas por Ativo**  
   Ranking de maiores oscilações diárias.

4. **⏰ Distribuição de Preço por Hora (Intradiário)**  
   Boxplot ou gráfico de dispersão para entender a volatilidade dentro do dia.

> ⚠️ As visualizações serão feitas via `display()` com gráficos interativos do Databricks. Nenhum dado será materializado em disco — todas as views serão temporárias.


In [0]:
from pyspark.sql.functions import hour, col, to_date, lag, round
from pyspark.sql.window import Window

In [0]:
catalogo = "lakehouse"
schema = "silver"
table = "cripto"
full_table_name = f"{catalogo}.{schema}.{table}"

# 📊 1. Evolução do Preço por Ativo ao Longo do Tempo

In [0]:
# View com média de preço diária por ativo
df_evolucao_diaria = (
    spark.read.table(f"{full_table_name}")
    .withColumn("date", to_date("timestamp"))
    .groupBy("asset_name", "date")
    .agg({"price_usd": "avg"})
    .withColumnRenamed("avg(price_usd)", "avg_price_usd")
)

df_evolucao_diaria.createOrReplaceTempView("vw_evolucao_preco_diario")

# Visualização
display(df_evolucao_diaria.orderBy("date"))

Databricks visualization. Run in Databricks to view.

#  📉 2. Variação Diária (%) por Ativo

In [0]:
janela = Window.partitionBy("asset_name").orderBy("date")

df_variacao = (
    df_evolucao_diaria
    .withColumn("lag_price", lag("avg_price_usd").over(janela))
    .withColumn("pct_change", round(((col("avg_price_usd") - col("lag_price")) / col("lag_price")) * 100, 2))
    .na.drop(subset=["pct_change"])
)

df_variacao.createOrReplaceTempView("vw_variacao_percentual_diaria")

display(df_variacao.orderBy("date"))

Databricks visualization. Run in Databricks to view.

# 🔥 3. Top 5 Maiores Altas e Quedas por Ativo

In [0]:
# Altas
top_altas = df_variacao.orderBy(col("pct_change").desc()).limit(5)
top_altas.createOrReplaceTempView("vw_top_5_altas")
display(top_altas)

# Quedas
top_quedas = df_variacao.orderBy(col("pct_change").asc()).limit(5)
top_quedas.createOrReplaceTempView("vw_top_5_quedas")
display(top_quedas)

# ⏰ 4. Distribuição de Preço por Hora (Intradiário)

In [0]:
df_intradiario = (
    spark.read.table(f"{full_table_name}")
    .filter("source_type = 'intradiario'")
    .withColumn("hora", hour("timestamp"))
)

df_intradiario.createOrReplaceTempView("vw_distribuicao_horaria_preco")

display(df_intradiario.select("asset_name", "hora", "price_usd"))


Databricks visualization. Run in Databricks to view.